In [1]:
import pandas as pd
import boto3
import json
import os
from typing import Dict, List

def initialize_bedrock_client():
    """Initialize the Bedrock client"""
    bedrock = boto3.client(
        service_name='bedrock-runtime',
        region_name='us-east-1'  # Adjust region as needed
    )
    return bedrock

def read_restaurant_data(file_path: str) -> pd.DataFrame:
    """Read the CSV file containing restaurant scores"""
    try:
        df = pd.read_csv(file_path)
        return df
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None

def calculate_category_scores(restaurant_data: Dict) -> Dict:
    """Calculate aggregate scores for main categories"""
    food_quality = (
        restaurant_data['cheese'] + 
        restaurant_data['crust'] + 
        restaurant_data['taste']
    ) / 3
    
    experience = (
        restaurant_data['environment'] + 
        restaurant_data['ambience'] + 
        restaurant_data['hygiene']
    ) / 3
    
    return {
        'food_quality': round(food_quality, 1),
        'experience': round(experience, 1)
    }

def generate_summary(bedrock_client, restaurant_data: Dict) -> str:
    """Generate a summary using Claude via Amazon Bedrock"""
    
    category_scores = calculate_category_scores(restaurant_data)
    
    prompt = f"""
    Please analyze this pizza restaurant data and create a concise, customer-friendly summary. 
    Focus on the key strengths and areas for improvement. Use the scores to provide specific insights.

    Restaurant Details:
    Name: {restaurant_data['business_id']}
    
    Food Scores:
    - Cheese Quality: {restaurant_data['cheese']}/10
    - Crust Quality: {restaurant_data['crust']}/10
    - Overall Taste: {restaurant_data['taste']}/10
    
    Service & Experience:
    - Delivery Speed: {restaurant_data['delivery_speed']}/10
    - Service Quality: {restaurant_data['service']}/10
    - Environment: {restaurant_data['environment']}/10
    - Ambience: {restaurant_data['ambience']}/10
    - Hygiene: {restaurant_data['hygiene']}/10
    - Overall Experience: {category_scores['experience']}/10
    
    Value:
    - Price Rating: {restaurant_data['price']}/10

    Please create a summary that includes:
    1. A brief overall impression (2-3 sentences)
    2. Key strengths (top 2-3 highest-rated aspects)
    3. Areas for improvement (2-3 lowest-rated aspects)
    4. Value assessment (considering price vs. quality)
    5. Best suited for (what type of dining experience, based on the scores)
    
    Make the summary engaging and helpful for customers deciding where to order pizza.
    """

    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 512,
        "temperature": 0.7,
        "messages": [{"role": "user", "content": prompt}]
    }

    try:
        response = bedrock_client.invoke_model(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            body=json.dumps(body)
        )
        response_body = json.loads(response['body'].read())
        return response_body['content'][0]['text']
    except Exception as e:
        print(f"Error generating summary: {str(e)}")
        return None

def analyze_restaurants(file_path: str):
    """Main function to analyze restaurants and generate summaries"""
    
    # Initialize Bedrock client
    bedrock_client = initialize_bedrock_client()
    
    # Read restaurant data
    df = read_restaurant_data(file_path)
    if df is None:
        return
    
    # Process each restaurant
    summaries = {}
    for idx, row in df.iterrows():
        restaurant_data = {
            'business_id': row['business_id'],
            'delivery_speed': row['delivery speed'],
            'service': row['service'],
            'cheese': row['cheese'],
            'crust': row['crust'],
            'taste': row['taste'],
            'price': row['price'],
            'environment': row['environment'],
            'ambience': row['ambience'],
            'hygiene': row['hygiene']
        }
        
        summary = generate_summary(bedrock_client, restaurant_data)
        if summary:
            summaries[restaurant_data['business_id']] = summary
    
    # Save summaries to a file
    output_file = 'pizza_restaurant_summaries.txt'
    with open(output_file, 'w') as f:
        for restaurant, summary in summaries.items():
            f.write(f"\n{'='*50}\n")
            f.write(f"Summary for {restaurant}\n")
            f.write(f"{'='*50}\n")
            f.write(summary)
            f.write("\n")
    
    print(f"Analysis complete. Summaries saved to {output_file}")

def generate_comparative_analysis(summaries: Dict[str, str]) -> str:
    """Generate a comparative analysis of all restaurants"""
    prompt = f"""
    Based on the summaries of all pizza restaurants, please provide:
    1. Top 3 overall restaurants
    2. Best value options
    3. Best for quick delivery
    4. Best dining experience
    5. Best pizza quality
    
    Make recommendations for different customer needs (e.g., family dining, quick lunch, date night, etc.)
    """
    
    # This could be implemented as another call to Claude
    pass

if __name__ == "__main__":
    csv_path = "./business_scores.csv"
    analyze_restaurants(csv_path)

Analysis complete. Summaries saved to pizza_restaurant_summaries.txt
